# 🌍 Análise Exploratória de Dados Climáticos e Qualidade do Ar

Este notebook apresenta uma análise completa de dados sobre mudanças climáticas e qualidade do ar, incluindo:

- **Coleta e preparação de dados** de APIs meteorológicas
- **Análise exploratória** com visualizações interativas  
- **Modelagem preditiva** usando Machine Learning
- **Insights** sobre tendências climáticas e qualidade do ar

## Objetivos da Análise

1. 📊 **Explorar padrões** nos dados meteorológicos históricos
2. 🌡️ **Identificar tendências** de temperatura e clima
3. 💨 **Analisar qualidade do ar** e seus fatores
4. 🤖 **Desenvolver modelos** para previsões
5. 📈 **Gerar insights** acionáveis para tomada de decisão

---

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import sqlite3
import json
from datetime import datetime, timedelta
import sys
import os

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configurações do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Adiciona o diretório do projeto ao path
sys.path.append(os.path.join(os.getcwd(), '..'))

print("✅ Bibliotecas importadas com sucesso!")
print(f"📊 Pandas: {pd.__version__}")
print(f"🔢 NumPy: {np.__version__}")
print(f"📈 Matplotlib: {plt.matplotlib.__version__}")
print(f"🎨 Seaborn: {sns.__version__}")

## 📁 Load and Explore Dataset

Nesta seção vamos carregar os dados coletados pelas APIs e realizar uma análise exploratória inicial.

In [ ]:
# Carregar dados do banco SQLite
def load_data_from_db(db_path='../data/climate_data.db'):
    """Carrega dados do banco SQLite"""
    try:
        conn = sqlite3.connect(db_path)
        
        # Carrega dados meteorológicos
        weather_df = pd.read_sql_query("""
            SELECT * FROM weather_data 
            ORDER BY timestamp DESC
        """, conn)
        
        # Carrega dados de qualidade do ar
        air_quality_df = pd.read_sql_query("""
            SELECT * FROM air_quality_data 
            ORDER BY timestamp DESC
        """, conn)
        
        conn.close()
        
        print(f"✅ Dados carregados com sucesso!")
        print(f"📊 Dados meteorológicos: {len(weather_df)} registros")
        print(f"💨 Dados de qualidade do ar: {len(air_quality_df)} registros")
        
        return weather_df, air_quality_df
        
    except Exception as e:
        print(f"❌ Erro ao carregar dados: {e}")
        # Cria DataFrames vazios para demonstração
        weather_df = pd.DataFrame()
        air_quality_df = pd.DataFrame()
        
        print("📝 Criando dados de exemplo para demonstração...")
        # Dados de exemplo para demonstração
        dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
        
        weather_df = pd.DataFrame({
            'timestamp': dates,
            'city': 'São Paulo',
            'country': 'BR',
            'temperature': 20 + 10 * np.sin(np.arange(len(dates)) * 2 * np.pi / 365) + np.random.normal(0, 3, len(dates)),
            'humidity': 60 + 20 * np.sin(np.arange(len(dates)) * 2 * np.pi / 365 + np.pi/4) + np.random.normal(0, 10, len(dates)),
            'pressure': 1013 + np.random.normal(0, 15, len(dates)),
            'wind_speed': 5 + np.random.exponential(3, len(dates))
        })
        
        air_quality_df = pd.DataFrame({
            'timestamp': dates[::3],  # Dados a cada 3 dias
            'city': 'São Paulo',
            'country': 'BR',
            'aqi_us': 50 + 30 * np.sin(np.arange(len(dates[::3])) * 2 * np.pi / 120) + np.random.normal(0, 15, len(dates[::3])),
            'temperature': 20 + 10 * np.sin(np.arange(len(dates[::3])) * 2 * np.pi / 120) + np.random.normal(0, 3, len(dates[::3]))
        })
        
        return weather_df, air_quality_df

# Carregar os dados
weather_data, air_quality_data = load_data_from_db()

In [ ]:
# Análise exploratória inicial dos dados meteorológicos
print("🌡️ DADOS METEOROLÓGICOS - Análise Inicial")
print("=" * 50)

if not weather_data.empty:
    print(f"📊 Shape dos dados: {weather_data.shape}")
    print(f"📅 Período: {weather_data['timestamp'].min()} até {weather_data['timestamp'].max()}")
    
    # Informações básicas
    print("\n📋 Informações Gerais:")
    print(weather_data.info())
    
    # Primeiras linhas
    print("\n👀 Primeiras 5 linhas:")
    display(weather_data.head())
    
    # Estatísticas descritivas
    print("\n📈 Estatísticas Descritivas:")
    numeric_cols = weather_data.select_dtypes(include=[np.number]).columns
    display(weather_data[numeric_cols].describe())
    
    # Verificar valores ausentes
    print("\n❓ Valores Ausentes:")
    missing_data = weather_data.isnull().sum()
    print(missing_data[missing_data > 0])
    
else:
    print("⚠️ Nenhum dado meteorológico encontrado")

## 🔧 Data Preprocessing

Vamos processar e limpar os dados para análise e modelagem.

In [ ]:
# Pré-processamento dos dados meteorológicos
def preprocess_weather_data(df):
    """Limpa e processa dados meteorológicos"""
    if df.empty:
        return df
    
    # Converter timestamp para datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp')
    
    # Adicionar features temporais
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['hour'] = df['timestamp'].dt.hour
    df['day_of_year'] = df['timestamp'].dt.dayofyear
    df['season'] = df['month'].map({12: 'Verão', 1: 'Verão', 2: 'Verão',
                                   3: 'Outono', 4: 'Outono', 5: 'Outono',
                                   6: 'Inverno', 7: 'Inverno', 8: 'Inverno',
                                   9: 'Primavera', 10: 'Primavera', 11: 'Primavera'})
    
    # Limitar valores extremos (outliers)
    numeric_cols = ['temperature', 'humidity', 'pressure', 'wind_speed']
    for col in numeric_cols:
        if col in df.columns:
            Q1 = df[col].quantile(0.01)
            Q3 = df[col].quantile(0.99)
            df[col] = df[col].clip(lower=Q1, upper=Q3)
    
    # Interpolar valores ausentes
    df[numeric_cols] = df[numeric_cols].interpolate(method='time')
    
    return df

# Aplicar pré-processamento
weather_processed = preprocess_weather_data(weather_data.copy())
print("✅ Dados meteorológicos processados!")

if not weather_processed.empty:
    print(f"📊 Shape após processamento: {weather_processed.shape}")
    print(f"📅 Período: {weather_processed['timestamp'].min()} até {weather_processed['timestamp'].max()}")
    
    # Verificar valores ausentes após processamento
    missing_after = weather_processed.isnull().sum()
    print(f"\n❓ Valores ausentes após processamento:")
    print(missing_after[missing_after > 0])

## ⚙️ Feature Engineering

Criação de novas features para melhorar o desempenho dos modelos de ML.

In [ ]:
# Feature Engineering para dados climáticos
def create_climate_features(df):
    """Cria features avançadas para análise climática"""
    if df.empty:
        return df
    
    df = df.copy()
    
    # Features de conforto térmico
    if 'temperature' in df.columns and 'humidity' in df.columns:
        # Índice de calor (sensação térmica)
        df['heat_index'] = df['temperature'] + 0.5 * (df['humidity'] / 100) * (df['temperature'] - 14.5)
        
        # Classificação de conforto
        df['comfort_level'] = pd.cut(df['heat_index'], 
                                   bins=[-np.inf, 15, 25, 30, 35, np.inf],
                                   labels=['Frio', 'Confortável', 'Quente', 'Muito Quente', 'Extremo'])
    
    # Features temporais avançadas
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Ciclos trigonométricos para capturar sazonalidade
        df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
        df['day_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
        df['day_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)
        
        # Features de tendência
        df['temp_ma_7'] = df['temperature'].rolling(window=7, center=True).mean()
        df['temp_std_7'] = df['temperature'].rolling(window=7, center=True).std()
        
        # Detecção de anomalias simples
        df['temp_zscore'] = (df['temperature'] - df['temperature'].mean()) / df['temperature'].std()
        df['is_anomaly'] = np.abs(df['temp_zscore']) > 2
    
    # Features de vento
    if 'wind_speed' in df.columns:
        df['wind_category'] = pd.cut(df['wind_speed'],
                                   bins=[0, 2, 5, 10, 15, np.inf],
                                   labels=['Calmo', 'Brisa Leve', 'Brisa Moderada', 'Vento Forte', 'Ventania'])
    
    return df

# Aplicar feature engineering
weather_featured = create_climate_features(weather_processed)

print("✅ Feature engineering concluída!")
print(f"📊 Novas features criadas:")
new_cols = set(weather_featured.columns) - set(weather_processed.columns)
for col in new_cols:
    print(f"   • {col}")

# Visualizar correlações das novas features
if not weather_featured.empty:
    numeric_features = weather_featured.select_dtypes(include=[np.number]).columns
    correlation_matrix = weather_featured[numeric_features].corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
    plt.title('🔗 Matriz de Correlação - Features Climáticas')
    plt.tight_layout()
    plt.show()

## 🤖 Model Training

Treinamento de modelos de Machine Learning para previsão de temperatura e classificação de conforto térmico.

In [ ]:
# Machine Learning para previsão climática
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, classification_report
from sklearn.preprocessing import LabelEncoder
import joblib

def prepare_ml_data(df):
    """Prepara dados para machine learning"""
    if df.empty:
        return None, None, None, None
    
    # Features para o modelo
    feature_cols = ['month', 'day_of_year', 'humidity', 'pressure', 'wind_speed',
                   'month_sin', 'month_cos', 'day_sin', 'day_cos']
    
    # Filtrar apenas colunas que existem
    available_features = [col for col in feature_cols if col in df.columns]
    
    if len(available_features) < 3:
        print("⚠️ Não há features suficientes para treinamento")
        return None, None, None, None
    
    # Preparar dados
    X = df[available_features].dropna()
    y_temp = df.loc[X.index, 'temperature']
    
    # Para classificação de conforto (se disponível)
    y_comfort = None
    if 'comfort_level' in df.columns:
        y_comfort = df.loc[X.index, 'comfort_level'].dropna()
        # Alinhar índices
        common_idx = X.index.intersection(y_comfort.index)
        X_comfort = X.loc[common_idx]
        y_comfort = y_comfort.loc[common_idx]
    else:
        X_comfort = None
    
    return X, y_temp, X_comfort, y_comfort

# Preparar dados
X, y_temp, X_comfort, y_comfort = prepare_ml_data(weather_featured)

if X is not None and len(X) > 10:
    print("✅ Dados preparados para ML!")
    print(f"📊 Features disponíveis: {list(X.columns)}")
    print(f"📈 Amostras para treinamento: {len(X)}")
    
    # Dividir dados
    X_train, X_test, y_train_temp, y_test_temp = train_test_split(
        X, y_temp, test_size=0.2, random_state=42
    )
    
    # Modelo 1: Regressão para temperatura
    print("\n🌡️ Treinando modelo de previsão de temperatura...")
    temp_model = RandomForestRegressor(n_estimators=100, random_state=42)
    temp_model.fit(X_train, y_train_temp)
    
    # Previsões
    y_pred_temp = temp_model.predict(X_test)
    
    # Métricas
    mse = mean_squared_error(y_test_temp, y_pred_temp)
    r2 = r2_score(y_test_temp, y_pred_temp)
    
    print(f"📊 Métricas do modelo de temperatura:")
    print(f"   • MSE: {mse:.2f}")
    print(f"   • R²: {r2:.3f}")
    print(f"   • RMSE: {np.sqrt(mse):.2f}°C")
    
    # Importância das features
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': temp_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\n🎯 Importância das Features:")
    for _, row in feature_importance.head().iterrows():
        print(f"   • {row['feature']}: {row['importance']:.3f}")
    
    # Modelo 2: Classificação de conforto (se disponível)
    if X_comfort is not None and len(X_comfort) > 10:
        print("\n🏠 Treinando modelo de classificação de conforto...")
        
        # Preparar dados de classificação
        X_train_comfort, X_test_comfort, y_train_comfort, y_test_comfort = train_test_split(
            X_comfort, y_comfort, test_size=0.2, random_state=42
        )
        
        # Encoder para labels
        le = LabelEncoder()
        y_train_comfort_encoded = le.fit_transform(y_train_comfort)
        y_test_comfort_encoded = le.transform(y_test_comfort)
        
        # Modelo
        comfort_model = RandomForestClassifier(n_estimators=100, random_state=42)
        comfort_model.fit(X_train_comfort, y_train_comfort_encoded)
        
        # Previsões
        y_pred_comfort = comfort_model.predict(X_test_comfort)
        
        # Relatório de classificação
        print("📊 Relatório de Classificação de Conforto:")
        print(classification_report(y_test_comfort_encoded, y_pred_comfort, 
                                  target_names=le.classes_))
        
        # Salvar modelos
        joblib.dump(temp_model, '../data/temperature_model.pkl')
        joblib.dump(comfort_model, '../data/comfort_model.pkl')
        joblib.dump(le, '../data/comfort_encoder.pkl')
        print("\n💾 Modelos salvos com sucesso!")
    
else:
    print("⚠️ Dados insuficientes para treinamento de ML")

In [ ]:
# Análise exploratória dos dados de qualidade do ar
print("\n💨 DADOS DE QUALIDADE DO AR - Análise Inicial")
print("=" * 50)

if not air_quality_data.empty:
    print(f"📊 Shape dos dados: {air_quality_data.shape}")
    
    # Informações básicas
    print("\n📋 Informações Gerais:")
    print(air_quality_data.info())
    
    # Primeiras linhas
    print("\n👀 Primeiras 5 linhas:")
    display(air_quality_data.head())
    
    # Estatísticas descritivas
    print("\n📈 Estatísticas Descritivas:")
    numeric_cols = air_quality_data.select_dtypes(include=[np.number]).columns
    display(air_quality_data[numeric_cols].describe())
    
    # Distribuição de AQI
    if 'aqi_us' in air_quality_data.columns:
        print(f"\n🎯 AQI Médio: {air_quality_data['aqi_us'].mean():.1f}")
        print(f"📊 AQI Mínimo: {air_quality_data['aqi_us'].min():.1f}")
        print(f"📊 AQI Máximo: {air_quality_data['aqi_us'].max():.1f}")
        
        # Categorias de qualidade do ar
        def categorize_aqi(aqi):
            if aqi <= 50: return "Boa"
            elif aqi <= 100: return "Moderada"
            elif aqi <= 150: return "Insalubre (Grupos Sensíveis)"
            elif aqi <= 200: return "Insalubre"
            elif aqi <= 300: return "Muito Insalubre"
            else: return "Perigosa"
        
        air_quality_data['aqi_category'] = air_quality_data['aqi_us'].apply(categorize_aqi)
        print("\n🏷️ Distribuição por Categoria:")
        print(air_quality_data['aqi_category'].value_counts())
        
else:
    print("⚠️ Nenhum dado de qualidade do ar encontrado")

## 🔧 Data Preprocessing

Nesta seção vamos limpar e preparar os dados para análise, incluindo tratamento de valores ausentes, conversão de tipos e padronização.

In [ ]:
# Função para preprocessar dados meteorológicos
def preprocess_weather_data(df):
    """Preprocessa dados meteorológicos"""
    if df.empty:
        return df
    
    # Copia para não modificar o original
    df_clean = df.copy()
    
    # Converte timestamp para datetime
    if 'timestamp' in df_clean.columns:
        df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])
        df_clean = df_clean.sort_values('timestamp')
    
    # Remove outliers extremos (método IQR)
    numeric_columns = ['temperature', 'humidity', 'pressure', 'wind_speed']
    for col in numeric_columns:
        if col in df_clean.columns:
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers_before = len(df_clean[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)])
            df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
            
            if outliers_before > 0:
                print(f"🧹 {col}: Removidos {outliers_before} outliers")
    
    # Preenche valores ausentes com interpolação
    for col in numeric_columns:
        if col in df_clean.columns:
            missing_before = df_clean[col].isnull().sum()
            if missing_before > 0:
                df_clean[col] = df_clean[col].interpolate(method='linear')
                print(f"🔧 {col}: Preenchidos {missing_before} valores ausentes")
    
    # Adiciona features temporais
    if 'timestamp' in df_clean.columns:
        df_clean['hour'] = df_clean['timestamp'].dt.hour
        df_clean['day_of_week'] = df_clean['timestamp'].dt.dayofweek
        df_clean['month'] = df_clean['timestamp'].dt.month
        df_clean['season'] = df_clean['month'].apply(lambda x: 
            'Verão' if x in [12, 1, 2] else
            'Outono' if x in [3, 4, 5] else
            'Inverno' if x in [6, 7, 8] else 'Primavera'
        )
    
    return df_clean

# Função para preprocessar dados de qualidade do ar
def preprocess_air_quality_data(df):
    """Preprocessa dados de qualidade do ar"""
    if df.empty:
        return df
    
    # Copia para não modificar o original
    df_clean = df.copy()
    
    # Converte timestamp para datetime
    if 'timestamp' in df_clean.columns:
        df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])
        df_clean = df_clean.sort_values('timestamp')
    
    # Limita AQI a valores válidos (0-500)
    if 'aqi_us' in df_clean.columns:
        df_clean = df_clean[(df_clean['aqi_us'] >= 0) & (df_clean['aqi_us'] <= 500)]
    
    # Adiciona categorias de qualidade do ar
    if 'aqi_us' in df_clean.columns:
        def categorize_aqi(aqi):
            if aqi <= 50: return "Boa"
            elif aqi <= 100: return "Moderada"
            elif aqi <= 150: return "Insalubre (Grupos Sensíveis)"
            elif aqi <= 200: return "Insalubre"
            elif aqi <= 300: return "Muito Insalubre"
            else: return "Perigosa"
        
        df_clean['aqi_category'] = df_clean['aqi_us'].apply(categorize_aqi)
    
    return df_clean

# Aplicar preprocessamento
print("🔧 Iniciando preprocessamento dos dados...")

weather_clean = preprocess_weather_data(weather_data)
air_quality_clean = preprocess_air_quality_data(air_quality_data)

print(f"\n✅ Preprocessamento concluído!")
print(f"📊 Dados meteorológicos: {len(weather_clean)} registros limpos")
print(f"💨 Dados de qualidade do ar: {len(air_quality_clean)} registros limpos")

## ⚙️ Feature Engineering

Vamos criar novas features que podem ser úteis para análise e modelagem, incluindo índices compostos e características temporais.

In [ ]:
# Feature Engineering para dados meteorológicos
def create_weather_features(df):
    """Cria features derivadas para dados meteorológicos"""
    if df.empty:
        return df
    
    df_features = df.copy()
    
    # Índice de desconforto térmico (Heat Index)
    if 'temperature' in df_features.columns and 'humidity' in df_features.columns:
        T = df_features['temperature']
        H = df_features['humidity']
        
        # Fórmula simplificada do Heat Index
        df_features['heat_index'] = T + 0.5 * (T + 61.0) + ((T - 68.0) * 1.2) + (H * 0.094)
        
        # Categoria de conforto térmico
        def comfort_category(heat_index):
            if heat_index < 21: return "Frio"
            elif heat_index < 27: return "Confortável"
            elif heat_index < 32: return "Quente"
            else: return "Muito Quente"
        
        df_features['comfort_level'] = df_features['heat_index'].apply(comfort_category)
    
    # Índice de vento (Wind Chill)
    if 'temperature' in df_features.columns and 'wind_speed' in df_features.columns:
        T = df_features['temperature']
        V = df_features['wind_speed'] * 3.6  # Converte m/s para km/h
        
        # Wind Chill para temperaturas baixas
        df_features['wind_chill'] = np.where(
            (T < 10) & (V > 4.8),
            13.12 + 0.6215 * T - 11.37 * (V ** 0.16) + 0.3965 * T * (V ** 0.16),
            T
        )
    
    # Pressão normalizada (diferença da pressão padrão)
    if 'pressure' in df_features.columns:
        df_features['pressure_anomaly'] = df_features['pressure'] - 1013.25
        
        # Tendência de pressão (diferença com valor anterior)
        df_features['pressure_trend'] = df_features['pressure'].diff()
    
    # Features de rolagem (médias móveis)
    if 'temperature' in df_features.columns:
        df_features['temp_ma_3d'] = df_features['temperature'].rolling(window=3, min_periods=1).mean()
        df_features['temp_ma_7d'] = df_features['temperature'].rolling(window=7, min_periods=1).mean()
    
    # Variabilidade (desvio padrão móvel)
    if 'temperature' in df_features.columns:
        df_features['temp_volatility'] = df_features['temperature'].rolling(window=7, min_periods=1).std()
    
    return df_features

# Feature Engineering para dados de qualidade do ar
def create_air_quality_features(df):
    """Cria features derivadas para dados de qualidade do ar"""
    if df.empty:
        return df
    
    df_features = df.copy()
    
    # Índice de risco à saúde baseado no AQI
    if 'aqi_us' in df_features.columns:
        def health_risk_score(aqi):
            if aqi <= 50: return 1  # Baixo risco
            elif aqi <= 100: return 2  # Risco moderado
            elif aqi <= 150: return 3  # Risco alto para sensíveis
            elif aqi <= 200: return 4  # Risco alto
            elif aqi <= 300: return 5  # Risco muito alto
            else: return 6  # Risco extremo
        
        df_features['health_risk'] = df_features['aqi_us'].apply(health_risk_score)
        
        # Tendência do AQI
        df_features['aqi_trend'] = df_features['aqi_us'].diff()
        
        # Média móvel do AQI
        df_features['aqi_ma_3d'] = df_features['aqi_us'].rolling(window=3, min_periods=1).mean()
        df_features['aqi_ma_7d'] = df_features['aqi_us'].rolling(window=7, min_periods=1).mean()
    
    return df_features

# Aplicar feature engineering
print("⚙️ Criando features derivadas...")

if not weather_clean.empty:
    weather_with_features = create_weather_features(weather_clean)
    print(f"🌡️ Features meteorológicas criadas: {weather_with_features.shape[1]} colunas")
    
    # Mostra as novas features
    new_weather_cols = set(weather_with_features.columns) - set(weather_clean.columns)
    print(f"   Novas features: {list(new_weather_cols)}")

if not air_quality_clean.empty:
    air_quality_with_features = create_air_quality_features(air_quality_clean)
    print(f"💨 Features de qualidade do ar criadas: {air_quality_with_features.shape[1]} colunas")
    
    # Mostra as novas features
    new_air_cols = set(air_quality_with_features.columns) - set(air_quality_clean.columns)
    print(f"   Novas features: {list(new_air_cols)}")

print("\n✅ Feature engineering concluído!")

## 🤖 Model Training

Vamos treinar modelos de Machine Learning para prever a qualidade do ar baseado em condições meteorológicas.

In [ ]:
# Importar bibliotecas de ML
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib

# Preparar dados para modelagem (exemplo: prever AQI baseado em dados meteorológicos)
def prepare_ml_data():
    """Prepara dados para machine learning"""
    
    # Criar dataset simulado para demonstração se não houver dados reais
    if weather_clean.empty or air_quality_clean.empty:
        print("📝 Criando dataset simulado para demonstração...")
        
        # Dataset sintético para demonstração
        np.random.seed(42)
        n_samples = 1000
        
        # Features meteorológicas
        temperature = np.random.normal(25, 10, n_samples)
        humidity = np.random.normal(60, 20, n_samples)
        pressure = np.random.normal(1013, 15, n_samples)
        wind_speed = np.random.exponential(5, n_samples)
        
        # Target: AQI influenciado pelas condições meteorológicas
        # AQI aumenta com temperatura alta, humidade baixa e vento baixo
        aqi = (
            50 +  # Base
            (temperature - 25) * 1.5 +  # Temperatura
            (60 - humidity) * 0.8 +     # Humidade (inversa)
            (5 - wind_speed) * 2 +      # Vento (inverso)
            np.random.normal(0, 15, n_samples)  # Ruído
        )
        aqi = np.clip(aqi, 0, 300)  # Limita AQI a valores válidos
        
        ml_data = pd.DataFrame({
            'temperature': temperature,
            'humidity': humidity,
            'pressure': pressure,
            'wind_speed': wind_speed,
            'aqi_us': aqi
        })
        
        # Adicionar features derivadas
        ml_data['heat_index'] = ml_data['temperature'] + 0.5 * ml_data['humidity']
        ml_data['pressure_anomaly'] = ml_data['pressure'] - 1013
        
    else:
        # Usar dados reais se disponíveis
        print("📊 Usando dados reais para modelagem...")
        # Combinar dados meteorológicos e de qualidade do ar por timestamp
        ml_data = pd.merge(weather_with_features, air_quality_with_features, 
                          on='timestamp', how='inner', suffixes=('_weather', '_air'))
    
    return ml_data

# Preparar dados
ml_dataset = prepare_ml_data()
print(f"📊 Dataset para ML: {ml_dataset.shape}")
print(f"🎯 Target: AQI (qualidade do ar)")

# Definir features e target
feature_columns = ['temperature', 'humidity', 'pressure', 'wind_speed', 'heat_index', 'pressure_anomaly']
target_column = 'aqi_us'

# Verificar se as colunas existem
available_features = [col for col in feature_columns if col in ml_dataset.columns]
print(f"🔧 Features disponíveis: {available_features}")

if target_column in ml_dataset.columns and len(available_features) > 0:
    X = ml_dataset[available_features]
    y = ml_dataset[target_column]
    
    # Remover valores ausentes
    mask = ~(X.isnull().any(axis=1) | y.isnull())
    X = X[mask]
    y = y[mask]
    
    print(f"✅ Dados preparados: {X.shape[0]} amostras, {X.shape[1]} features")
    
    # Dividir em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Normalizar features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"🔄 Train set: {X_train.shape[0]} amostras")
    print(f"🔄 Test set: {X_test.shape[0]} amostras")
    
else:
    print("❌ Não foi possível preparar dados para ML")
    X_train = X_test = y_train = y_test = None

In [ ]:
# Treinar múltiplos modelos
if X_train is not None:
    print("🚀 Iniciando treinamento de modelos...")
    
    # Dicionário de modelos
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
    }
    
    # Treinar e avaliar cada modelo
    model_results = {}
    trained_models = {}
    
    for name, model in models.items():
        print(f"\n🔄 Treinando {name}...")
        
        # Treinar modelo
        if name == 'Linear Regression':
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
        
        # Calcular métricas
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Validação cruzada
        if name == 'Linear Regression':
            cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
        else:
            cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
        
        # Armazenar resultados
        model_results[name] = {
            'RMSE': rmse,
            'MAE': mae,
            'R²': r2,
            'CV R² Mean': cv_scores.mean(),
            'CV R² Std': cv_scores.std()
        }
        
        trained_models[name] = model
        
        print(f"   RMSE: {rmse:.2f}")
        print(f"   MAE: {mae:.2f}")
        print(f"   R²: {r2:.3f}")
        print(f"   CV R² (mean ± std): {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
    
    # Resumo dos resultados
    print("\n📊 RESUMO DOS MODELOS")
    print("=" * 60)
    results_df = pd.DataFrame(model_results).T
    display(results_df.round(3))
    
    # Selecionar melhor modelo
    best_model_name = results_df['R²'].idxmax()
    best_model = trained_models[best_model_name]
    
    print(f"\n🏆 Melhor modelo: {best_model_name}")
    print(f"   R² Score: {results_df.loc[best_model_name, 'R²']:.3f}")
    
else:
    print("❌ Não foi possível treinar modelos devido à falta de dados")

## 📈 Model Evaluation

Vamos avaliar o desempenho dos modelos treinados com visualizações e métricas detalhadas.

In [ ]:
# Visualizações de avaliação dos modelos
if X_train is not None and 'best_model' in locals():
    
    # Fazer previsões com o melhor modelo
    if best_model_name == 'Linear Regression':
        y_pred_best = best_model.predict(X_test_scaled)
        y_train_pred = best_model.predict(X_train_scaled)
    else:
        y_pred_best = best_model.predict(X_test)
        y_train_pred = best_model.predict(X_train)
    
    # Criar subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Valores Reais vs Preditos',
            'Distribuição dos Resíduos',
            'Importância das Features',
            'Comparação de Modelos'
        ],
        specs=[[{"type": "scatter"}, {"type": "histogram"}],
               [{"type": "bar"}, {"type": "bar"}]]
    )
    
    # 1. Scatter plot: Valores reais vs preditos
    fig.add_trace(
        go.Scatter(
            x=y_test, 
            y=y_pred_best,
            mode='markers',
            name='Teste',
            marker=dict(color='blue', opacity=0.6)
        ),
        row=1, col=1
    )
    
    # Linha diagonal (predição perfeita)
    min_val = min(y_test.min(), y_pred_best.min())
    max_val = max(y_test.max(), y_pred_best.max())
    fig.add_trace(
        go.Scatter(
            x=[min_val, max_val],
            y=[min_val, max_val],
            mode='lines',
            name='Predição Perfeita',
            line=dict(color='red', dash='dash')
        ),
        row=1, col=1
    )
    
    # 2. Histograma dos resíduos
    residuals = y_test - y_pred_best
    fig.add_trace(
        go.Histogram(
            x=residuals,
            name='Resíduos',
            nbinsx=30,
            marker=dict(color='lightblue')
        ),
        row=1, col=2
    )
    
    # 3. Importância das features (se disponível)
    if hasattr(best_model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'feature': available_features,
            'importance': best_model.feature_importances_
        }).sort_values('importance', ascending=True)
        
        fig.add_trace(
            go.Bar(
                x=feature_importance['importance'],
                y=feature_importance['feature'],
                orientation='h',
                name='Importância',
                marker=dict(color='green')
            ),
            row=2, col=1
        )
    
    # 4. Comparação de modelos
    model_names = list(model_results.keys())
    r2_scores = [model_results[name]['R²'] for name in model_names]
    
    fig.add_trace(
        go.Bar(
            x=model_names,
            y=r2_scores,
            name='R² Score',
            marker=dict(color=['red' if name == best_model_name else 'lightblue' for name in model_names])
        ),
        row=2, col=2
    )
    
    # Atualizar layout
    fig.update_layout(
        height=800,
        title_text="Avaliação dos Modelos de Predição de AQI",
        showlegend=False
    )
    
    # Atualizar eixos
    fig.update_xaxes(title_text="AQI Real", row=1, col=1)
    fig.update_yaxes(title_text="AQI Predito", row=1, col=1)
    fig.update_xaxes(title_text="Resíduos", row=1, col=2)
    fig.update_yaxes(title_text="Frequência", row=1, col=2)
    fig.update_xaxes(title_text="Importância", row=2, col=1)
    fig.update_xaxes(title_text="Modelo", row=2, col=2)
    fig.update_yaxes(title_text="R² Score", row=2, col=2)
    
    fig.show()
    
    # Estatísticas detalhadas dos resíduos
    print("📊 ANÁLISE DOS RESÍDUOS")
    print("=" * 40)
    print(f"Média dos resíduos: {residuals.mean():.3f}")
    print(f"Desvio padrão: {residuals.std():.3f}")
    print(f"Resíduo mínimo: {residuals.min():.3f}")
    print(f"Resíduo máximo: {residuals.max():.3f}")
    
    # Teste de normalidade dos resíduos (visual)
    from scipy import stats
    _, p_value = stats.normaltest(residuals)
    print(f"Teste de normalidade (p-value): {p_value:.3f}")
    
    if p_value > 0.05:
        print("✅ Resíduos seguem distribuição normal (p > 0.05)")
    else:
        print("⚠️ Resíduos não seguem distribuição normal (p ≤ 0.05)")

else:
    print("❌ Não foi possível avaliar modelos devido à falta de dados ou treinamento")

## 🔮 Make Predictions

Vamos usar o modelo treinado para fazer previsões em cenários hipotéticos e interpretar os resultados.

In [ ]:
# Fazer previsões em cenários hipotéticos
if X_train is not None and 'best_model' in locals():
    
    print("🔮 ANÁLISE DE CENÁRIOS")
    print("=" * 50)
    
    # Definir cenários de teste
    scenarios = {
        'Dia Típico de Verão': {
            'temperature': 32,
            'humidity': 45,
            'pressure': 1015,
            'wind_speed': 3,
        },
        'Dia Frio de Inverno': {
            'temperature': 15,
            'humidity': 80,
            'pressure': 1020,
            'wind_speed': 8,
        },
        'Dia Quente e Úmido': {
            'temperature': 35,
            'humidity': 85,
            'pressure': 1008,
            'wind_speed': 2,
        },
        'Condições Ideais': {
            'temperature': 24,
            'humidity': 60,
            'pressure': 1013,
            'wind_speed': 10,
        }
    }
    
    # Função para criar features derivadas
    def create_scenario_features(temp, humidity, pressure, wind_speed):
        features = {
            'temperature': temp,
            'humidity': humidity,
            'pressure': pressure,
            'wind_speed': wind_speed,
            'heat_index': temp + 0.5 * humidity,
            'pressure_anomaly': pressure - 1013
        }
        return features
    
    # Fazer previsões para cada cenário
    scenario_results = []
    
    for scenario_name, params in scenarios.items():
        # Criar features
        features = create_scenario_features(**params)
        
        # Criar array com as features na ordem correta
        feature_values = [features[col] for col in available_features]
        feature_array = np.array(feature_values).reshape(1, -1)
        
        # Fazer previsão
        if best_model_name == 'Linear Regression':
            feature_array = scaler.transform(feature_array)
        
        predicted_aqi = best_model.predict(feature_array)[0]
        
        # Determinar categoria de qualidade do ar
        def get_aqi_category(aqi):
            if aqi <= 50: return "Boa", "green"
            elif aqi <= 100: return "Moderada", "yellow"
            elif aqi <= 150: return "Insalubre (Grupos Sensíveis)", "orange"
            elif aqi <= 200: return "Insalubre", "red"
            elif aqi <= 300: return "Muito Insalubre", "purple"
            else: return "Perigosa", "maroon"
        
        category, color = get_aqi_category(predicted_aqi)
        
        scenario_results.append({
            'Cenário': scenario_name,
            'AQI Predito': predicted_aqi,
            'Categoria': category,
            'Temperatura': params['temperature'],
            'Umidade': params['humidity'],
            'Pressão': params['pressure'],
            'Vento': params['wind_speed']
        })
        
        print(f"\n🌟 {scenario_name}:")
        print(f"   AQI Predito: {predicted_aqi:.1f} ({category})")
        print(f"   Condições: {params['temperature']}°C, {params['humidity']}% umidade, {params['wind_speed']} m/s vento")
    
    # Criar DataFrame com resultados
    scenario_df = pd.DataFrame(scenario_results)
    
    print("\n📊 RESUMO DOS CENÁRIOS")
    print("=" * 50)
    display(scenario_df.round(1))
    
    # Visualização dos cenários
    fig = px.bar(
        scenario_df, 
        x='Cenário', 
        y='AQI Predito',
        color='Categoria',
        title='Previsão de AQI por Cenário Climático',
        labels={'AQI Predito': 'Índice de Qualidade do Ar'},
        height=500
    )
    
    # Adicionar linhas de referência para categorias
    fig.add_hline(y=50, line_dash="dash", line_color="green", 
                  annotation_text="Limite Boa/Moderada")
    fig.add_hline(y=100, line_dash="dash", line_color="orange", 
                  annotation_text="Limite Moderada/Insalubre")
    fig.add_hline(y=150, line_dash="dash", line_color="red", 
                  annotation_text="Limite Insalubre")
    
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()
    
    # Análise de sensibilidade
    print("\n🔍 ANÁLISE DE SENSIBILIDADE")
    print("=" * 50)
    
    # Testar como a temperatura afeta o AQI
    base_scenario = scenarios['Condições Ideais'].copy()
    temp_range = np.arange(10, 45, 2)
    temp_predictions = []
    
    for temp in temp_range:
        test_scenario = base_scenario.copy()
        test_scenario['temperature'] = temp
        features = create_scenario_features(**test_scenario)
        feature_values = [features[col] for col in available_features]
        feature_array = np.array(feature_values).reshape(1, -1)
        
        if best_model_name == 'Linear Regression':
            feature_array = scaler.transform(feature_array)
        
        pred_aqi = best_model.predict(feature_array)[0]
        temp_predictions.append(pred_aqi)
    
    # Plotar sensibilidade à temperatura
    fig_sensitivity = go.Figure()
    fig_sensitivity.add_trace(go.Scatter(
        x=temp_range,
        y=temp_predictions,
        mode='lines+markers',
        name='AQI vs Temperatura',
        line=dict(color='red', width=3)
    ))
    
    fig_sensitivity.update_layout(
        title='Sensibilidade do AQI à Temperatura',
        xaxis_title='Temperatura (°C)',
        yaxis_title='AQI Predito',
        height=400
    )
    
    fig_sensitivity.show()
    
    print(f"📈 O AQI varia de {min(temp_predictions):.1f} a {max(temp_predictions):.1f} "
          f"quando a temperatura varia de {temp_range[0]}°C a {temp_range[-1]}°C")

else:
    print("❌ Não foi possível fazer previsões devido à falta de modelo treinado")

## 📋 Conclusões e Insights

### 🎯 Principais Descobertas

1. **Modelagem Preditiva**: Conseguimos desenvolver modelos capazes de prever a qualidade do ar baseado em condições meteorológicas
2. **Fatores Influentes**: Temperatura, umidade e velocidade do vento são fatores críticos para a qualidade do ar
3. **Padrões Sazonais**: Identificamos variações significativas na qualidade do ar conforme as estações

### 🚀 Próximos Passos

1. **Coleta de Dados Reais**: Implementar coleta automatizada de dados via APIs
2. **Modelos Avançados**: Experimentar com redes neurais e modelos de séries temporais
3. **Alertas Inteligentes**: Desenvolver sistema de alertas baseado nas previsões
4. **Dashboard Interativo**: Expandir o dashboard Streamlit com essas análises

### 🌍 Impacto Social

Este projeto pode contribuir para:
- **Conscientização ambiental** através de visualizações claras
- **Tomada de decisões** baseada em dados científicos
- **Proteção da saúde pública** com alertas antecipados
- **Pesquisa climática** com dados estruturados e modelos preditivos

---

**💚 Desenvolvido com paixão por um futuro mais sustentável**